In [26]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.linalg import hankel

In [27]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [28]:
L = 50
stride = 0.02

In [29]:
df = pd.read_csv('~/data/ctown/dataset03.csv')
df

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,05/01/15 20,3.349998,3.393529,3.704372,3.318569,1.976920,5.500000,4.797106,96.594833,1,...,71.224411,28.940384,84.560669,21.076448,81.414070,20.970821,71.399384,31.532953,30.632954,0
8757,05/01/15 21,3.262380,3.738153,3.319207,3.162533,2.355615,5.207708,3.493474,96.445862,1,...,69.354080,29.247887,84.673790,26.999941,83.265991,26.999941,52.213036,31.836996,30.936996,0
8758,05/01/15 22,3.114685,3.948144,3.123156,3.060679,2.593729,4.997561,2.207609,96.929070,1,...,86.823753,28.358240,84.593727,20.571791,81.679344,20.457272,66.498741,31.281141,30.381142,0
8759,05/01/15 23,2.931601,3.892631,3.542954,3.025374,2.923712,5.104730,2.071863,97.206474,1,...,87.819832,28.297560,83.987007,20.450956,82.001610,20.337372,66.844086,31.210789,30.310789,0


In [30]:
df_1 = pd.read_csv('~/data/ctown/dataset04.csv')
df_normal=df_1[df_1['ATT_FLAG']==0]



In [31]:
df=pd.concat((df,df_normal),axis=0,ignore_index=True)
df

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,24/12/16 20,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,0
12442,24/12/16 21,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,0
12443,24/12/16 22,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,0
12444,24/12/16 23,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,0


In [32]:
df=df.drop(['DATETIME'],axis=1)
df_label=df.iloc[:,-1]
df=df.iloc[:,:-1]
df_colms=df.columns
scaler=StandardScaler()
df=scaler.fit_transform(df)
df=pd.DataFrame(df,columns=df_colms)
df=pd.concat((df,df_label),axis=1)
df

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477,0
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926,0
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820,0
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060,0
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880,0
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695,0
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839,0
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194,0


# --Done with training data--

In [33]:
df_attack=pd.read_csv('/home/gajender/meepc/attackdata_labeledwithattacktype.csv')
df_attack

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1,93.21,...,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1,1
1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1,91.46,...,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1,1
2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1,90.24,...,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1,1
3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1,94.79,...,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1,1
4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1,94.81,...,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.39,3.81,4.02,3.28,4.00,5.50,3.77,98.55,1,98.57,...,28.10,86.71,19.27,82.66,19.16,70.54,30.92,30.02,1,7
488,0.50,3.69,4.50,3.72,3.31,5.50,4.27,97.42,1,97.44,...,28.09,85.77,26.08,62.67,25.97,73.59,30.95,30.05,1,7
489,0.66,3.61,4.97,3.90,2.61,5.50,4.95,96.87,1,96.89,...,28.10,87.05,29.10,62.90,29.10,60.38,30.96,30.06,1,7
490,0.92,3.68,5.33,4.31,2.08,5.22,3.65,95.94,1,95.96,...,29.24,89.16,29.62,62.82,29.62,59.34,31.80,30.90,1,7


In [34]:
df_attack['AttackID'].value_counts()

7    110
4     94
6     94
3     60
5     60
1     50
2     24
Name: AttackID, dtype: int64

In [35]:
df_test=pd.read_csv('/home/gajender/meepc/testdata_labeledwithattacktype.csv')
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,98.95,...,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0,-1
1,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,97.97,...,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0,-1
2,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,96.84,...,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0,-1
3,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,96.78,...,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0,-1
4,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,94.79,...,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2085,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2086,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2087,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [36]:
test_colms=df_test.columns

In [37]:
df_test['AttackID'].value_counts()

-1     1682
 12     100
 13      80
 8       70
 9       65
 10      31
 11      31
 14      30
Name: AttackID, dtype: int64

In [38]:
df_test_combined=pd.concat((df_attack,df_test),axis=0,ignore_index=True)
df_test_combined

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1.0,93.21,...,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1.0,1
1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1.0,91.46,...,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1.0,1
2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1.0,90.24,...,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1.0,1
3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1.0,94.79,...,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1.0,1
4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1.0,94.81,...,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2577,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2578,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2579,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [39]:
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,98.95,...,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0,-1
1,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,97.97,...,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0,-1
2,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,96.84,...,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0,-1
3,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,96.78,...,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0,-1
4,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,94.79,...,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2085,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2086,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2087,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [40]:
y_actual=df_test.iloc[:,-1].values
# attacklabels = hankel(y_actual[:L],y_actual[L-1:])
attacklabels=hankel.fit(np.asarray(y_actual),50,0.5)
attacklabels

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [41]:
attacklabels.shape

(50, 82)

In [42]:
actual_labels=np.any(attacklabels>0,axis=0).astype(int)
isattack = np.any(attacklabels>0,axis=0)
attackindices=np.where(isattack)[0]
attackindices

array([10, 11, 12, 13, 14, 24, 25, 26, 27, 33, 34, 35, 36, 37, 38, 48, 49,
       50, 51, 52, 53, 61, 62, 63, 64, 65, 66, 76, 77, 78])

In [43]:
actual_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [44]:
attack_dict={}
for idx in attackindices:
    attack_dict[idx]=np.unique(attacklabels[:,idx])
attack_dict

{10: array([-1,  8]),
 11: array([-1,  8]),
 12: array([8]),
 13: array([-1,  8]),
 14: array([-1,  8]),
 24: array([-1,  9]),
 25: array([-1,  9]),
 26: array([-1,  9]),
 27: array([-1,  9]),
 33: array([-1, 10]),
 34: array([-1, 10]),
 35: array([-1, 10]),
 36: array([-1, 11]),
 37: array([-1, 11]),
 38: array([-1, 11]),
 48: array([-1, 12]),
 49: array([-1, 12]),
 50: array([12]),
 51: array([12]),
 52: array([-1, 12]),
 53: array([-1, 12]),
 61: array([-1, 13]),
 62: array([-1, 13]),
 63: array([13]),
 64: array([13]),
 65: array([-1, 13]),
 66: array([-1, 13]),
 76: array([-1, 14]),
 77: array([-1, 14]),
 78: array([-1, 14])}

In [45]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    idx = np.argmax(fmeas)
    return radiis[indices[idx]]

In [46]:
df_test.columns

Index(['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1',
       'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4', 'F_PU5',
       'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8', 'F_PU9',
       'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2', 'S_V2',
       'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415', 'P_J302',
       'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422', 'ATT_FLAG',
       'AttackID'],
      dtype='object')

In [47]:
df_normal=df
df_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477,0
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926,0
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820,0
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060,0
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880,0
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695,0
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839,0
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194,0


In [48]:
df_test.drop(columns=['ATT_FLAG','AttackID'],inplace=True)
test_cols=df_test.columns
scaler=StandardScaler()
df_test=scaler.fit_transform(df_test)
df_test=pd.DataFrame(df_test,columns=test_cols)
df_test


,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,-1.691904,-0.752059,-0.293836,-0.571260,1.516235,0.726698,1.204044,-0.109271,0.069354,0.735521,...,-0.449652,1.281197,-0.435599,0.215417,-1.142721,0.890069,-1.150464,0.648516,-0.994371,-0.346750
1,-1.724926,-0.765939,0.489027,-0.571260,1.475512,0.726698,1.815205,-0.189570,0.069354,0.712890,...,-0.453103,1.268529,-0.439075,0.283150,0.318342,-1.016702,0.312475,1.109080,-0.989524,-0.339299
2,-1.551562,-0.724300,1.227578,-0.295172,0.769642,-1.247388,-0.091619,-0.282158,0.069354,0.686794,...,-0.397882,1.417087,-0.383468,0.559292,1.035468,-1.057740,1.056266,-1.650556,-0.948316,-0.275972
3,-1.378199,-0.564682,1.419602,0.735555,0.090921,-2.118307,-0.922798,-0.287075,0.069354,0.685409,...,0.099115,-0.379426,0.082237,0.776818,0.443448,-1.096776,0.435693,0.571756,-0.705916,0.096541
4,-1.246112,-0.252387,1.094639,1.287731,-0.547077,0.029962,0.140623,-0.450129,0.069354,0.639454,...,1.151781,-0.430097,1.156139,-1.734508,0.651214,-1.095775,0.644044,0.564267,-0.024775,1.143304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,-0.693000,-0.876977,-0.781280,-1.141841,-1.375117,-2.408614,-1.472844,1.617139,0.069354,-1.549541,...,-0.273633,-1.076151,-0.293107,0.154197,-1.399635,0.601801,-1.410343,-0.354987,-0.982252,-0.328124
2085,-1.023216,-0.752059,-1.416434,-1.344305,-0.927161,-2.060246,-1.778424,1.799859,0.069354,-1.549541,...,-0.297792,-1.281138,-0.317435,0.033060,-1.676656,0.546750,-1.688144,-0.323160,-1.013763,-0.376551
2086,-1.411220,-0.772879,-1.785709,-2.338221,-0.560652,-1.479633,-1.851764,1.817065,0.069354,-1.549541,...,-0.639477,0.882740,-0.623271,0.117726,-1.692294,0.587788,-1.703827,-0.295077,-1.147083,-0.581433
2087,-1.592839,-0.842278,-1.121014,-2.246192,-0.112696,-0.782897,-1.802871,-0.127298,0.069354,0.730440,...,-0.501423,0.971414,-0.487730,0.156802,-1.238784,0.699892,-1.249039,-0.336265,-1.023459,-0.391451


In [49]:
attack_sensors = [0]*15
attack_sensors

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [78]:
def Epasad(df_normal,df_attack,df_test,clustering,flag):
    accuracy_=[]
    precision_=[]
    recall_=[]
    f1_=[]

    for sens in range(len(df_normal.columns)-1):
        X = df_normal.iloc[:,sens].values
        X = hankel.fit(X,50,0.5)
        X=X.T 
        if(clustering):
            kmeans,optimal_k = cluster.fit(X)
        else:
            optimal_k = 1
            kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
        kmeans.fit(X)
        
        radiis_normal = []
        weights=[]
        centers=[]
        clusters_R=[]
        clusters_V=[]
        for i in range(optimal_k):
            cluster_ = X[np.where(kmeans.labels_ == i)[0]]
            r = rank.fit(cluster_)
            clusters_R.append(r)
            U,Sigma,VT = np.linalg.svd(cluster_)
            V = VT.T
            clusters_V.append(V)
            cluster_ = np.matmul(cluster_,V[:,:r])
            weight,center = meepc.fit(cluster_)
            weights.append(weight)
            centers.append(center)
            var1=np.square(cluster_-center)
            var2=np.matmul(weight,var1.T)
            if(flag):
                radiis_normal.append((np.sqrt(var2)))
            else:
                radiis_normal.append(np.max((np.sqrt(var2))))

        if(flag):
            X_attack = df_attack.iloc[:,sens].values
            X_att = hankel.fit(X_attack,50,0.5)
            X_att=X_att.T 
            radiis_attack = []
            for i in range(optimal_k):
                cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
                var1=np.square(cluster_-centers[i])
                var2=np.matmul(weights[i],var1.T)
                radiis_attack.append(np.sqrt(var2))
            threshold_clusters = [0]*optimal_k
            for i in range(optimal_k):
                threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])


        X_test=df_test.iloc[:,sens].values
        Xtest=hankel.fit(X_test,50,0.5)  
        Xtest=Xtest.T  
        distances=[]
        for i in range(optimal_k):
            Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) 
            var1=np.square(Xtest_cluster-centers[i])
            var2=np.matmul(weights[i],var1.T)
            distance=np.sqrt(var2)
            distances.append(distance)

        stacked_arr=np.vstack(distances)
        distances_arr=np.transpose(stacked_arr)
    
       
        
        if (flag):
            check_anamoly=threshold_clusters-distances_arr    
            y_score=np.min((distances_arr/threshold_clusters),axis=1) 
        else:
            check_anamoly=radiis_normal-distances_arr
        
        predicted_labels=np.all(check_anamoly<0,axis=1).astype(int)
        is_positive=np.any(check_anamoly>0,axis=1)
        
        positive_indices=np.where(is_positive)[0]

        
        accuracy = accuracy_score(actual_labels, predicted_labels)
   
        precision = precision_score(actual_labels, predicted_labels)
        recall = recall_score(actual_labels, predicted_labels)
        f1 = f1_score(actual_labels, predicted_labels)
        if precision and recall and f1:
            accuracy_.append(accuracy)
            precision_.append(precision)
            recall_.append(recall)
            f1_.append(f1)
    print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy_).mean(), np.median(np.asarray(accuracy_)),np.asarray(accuracy_).min(), np.asarray(accuracy_).max()))
    print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision_).mean(), np.median(np.asarray(precision_)),np.asarray(precision_).min(), np.asarray(precision_).max()))
    print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall_).mean(), np.median(np.asarray(recall_)),np.asarray(recall_).min(), np.asarray(recall_).max()))
    print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(f1_).mean(), np.median(np.asarray(f1_)),np.asarray(f1_).min(), np.asarray(f1_).max()))

# 1) with 1 cluster

In [79]:
Epasad(df_normal,df_attack,df_test,0,0)

Accuracy -  Mean: 0.6565040650406503 Median : 0.6585365853658537 Min : 0.6219512195121951 Max : 0.7073170731707317
precision - Mean: 0.7593253968253969 Median : 1.0 Min : 0.3333333333333333 Max : 1.0
recall -    Mean: 0.09305555555555556 Median : 0.1 Min : 0.03333333333333333 Max : 0.2
f1 -        Mean: 0.16231624330533248 Median : 0.17424242424242425 Min : 0.0606060606060606 Max : 0.33333333333333337


# 2) with  multiple clusters

In [80]:
Epasad(df_normal,df_attack,df_test,1,0)

Accuracy -  Mean: 0.623524783634933 Median : 0.6341463414634146 Min : 0.43902439024390244 Max : 0.7073170731707317
precision - Mean: 0.49061166667703165 Median : 0.5 Min : 0.15384615384615385 Max : 1.0
recall -    Mean: 0.22365591397849466 Median : 0.2 Min : 0.03333333333333333 Max : 0.5
f1 -        Mean: 0.2920283662155316 Median : 0.28571428571428575 Min : 0.0606060606060606 Max : 0.5


# 3) with threshold tuning

In [81]:
Epasad(df_normal,df_attack,df_test,1,1)

Accuracy -  Mean: 0.6066089693154995 Median : 0.6097560975609756 Min : 0.43902439024390244 Max : 0.8292682926829268
precision - Mean: 0.43755111522963314 Median : 0.4166666666666667 Min : 0.15384615384615385 Max : 0.8888888888888888
recall -    Mean: 0.22043010752688172 Median : 0.2 Min : 0.03333333333333333 Max : 0.6666666666666666
f1 -        Mean: 0.2791957712077393 Median : 0.26086956521739135 Min : 0.05714285714285715 Max : 0.7407407407407408


In [ ]:
# cnt=0
        # # print("sensor ->",sens)
        # unique_s = set()
        # for i in range(len(Xtest)):
        #     if i not in positive_indices:
        #             if i in attack_dict:
        #                 for j in attack_dict[i]:
        #                     if j != -1:
        #                         unique_s.add(j)
        #             print(i,end=' ')
        #             if i in list(attackindices):
        #                 cnt+=1
        # # print("count",cnt)
        # print("\n attack detected out of those 14 attacks")
        # print(unique_s)
        # for i in unique_s:
        #     attack_sensors[i] += 1
        # # print(attack_sensors)
        # print("\nnext")
        # print(threshold_clusters)
        # print(distances_arr)